In [1]:
from __future__ import absolute_import, print_function
import torch
from torch import nn
import math
from torch.nn.parameter import Parameter
from torch.nn import functional as F
import numpy as np


In [32]:
#最终版
class MemModule1_new(nn.Module):
    def __init__(self,mem_dim,fea_dim,window=1,shrink_thres=0.0025, device='cuda'):
        super(MemModule1_new, self).__init__()
        self.mem_dim = mem_dim
        self.fea_dim = fea_dim
        self.shrink_thres = shrink_thres
        self.memory = MemoryUnit(self.mem_dim, self.fea_dim, self.shrink_thres)
        self.window=window

    def forward(self, x):
        s = x.data.shape
        window_size=s[2]//self.window
        if self.window!=1:
            
            x = x.view(s[0],s[1],self.window,window_size,self.window,window_size)
            print(x.shape)
            x=x.permute(0, 1, 2, 4,3,5).contiguous().view(-1, window_size*window_size)
            print(x.shape)
            y_and = self.memory(x)
            y = y_and['output']
            att = y_and['att']
            print(y.shape)
            y = y.view(s[0],s[1],self.window,self.window,window_size,window_size)
            print(y.shape)
            y=y.permute(0, 1, 2, 4,3,5).contiguous().view(s[0], s[1], s[2], s[3])
            print(y.shape)
            att = att.view(s[0],s[1],self.window,self.window,self.mem_dim)
            att = att.permute(0, 4, 1, 2, 3)
        else:
            x = x.view(s[0]*s[1],-1)
            print(x.shape)
            y_and = self.memory(x)
            y = y_and['output']
            att = y_and['att']
            print(att.shape)
    
            y = y.view(s[0],s[1],s[2],s[3])
            
            att = att.view(s[0],s[1],self.mem_dim)
            att = att.permute(0, 2,1)
            print(att.shape)
            
            
    #         att = att.view(s[0]* s[1],self.window, self.window, self.mem_dim)
        return {'output': y, 'att': att}


In [33]:
# window 窗口比例 fea_dim=w/window*h/window
model1=MemModule1_new(mem_dim=2000, fea_dim=256,window=2)
x1=torch.rand((2,256,32,32))
out1=model1(x1)

torch.Size([2, 256, 2, 16, 2, 16])
torch.Size([2048, 256])
torch.Size([2048, 256])
torch.Size([2, 256, 2, 2, 16, 16])
torch.Size([2, 256, 32, 32])


In [34]:
model1=MemModule1_new(mem_dim=2000, fea_dim=1024)
x1=torch.rand((2,256,32,32))
out1=model1(x1)

torch.Size([512, 1024])
torch.Size([512, 2000])
torch.Size([2, 2000, 256])


In [11]:
out1['output'].shape

torch.Size([2, 256, 32, 32])

In [37]:
def window_partition(x, window_size):
    B, H, W, C = x.shape
    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
#     print(x.shape)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C)
#     print(windows.shape)
    return windows


def window_reverse(windows, window_size, H, W):
    B = int(windows.shape[0] / (H * W / window_size / window_size))
    x = windows.view(B, H // window_size, W // window_size, window_size, window_size, -1)
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(B, H, W, -1)
    return x


In [62]:
x=torch.rand((2,32,32,128))
windows=window_partition(x,2)
y=window_reverse(windows,2,32,32)

In [38]:
def window_partition_c(x, window_size,c_size):
    B, H, W, C = x.shape
    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C//c_size,c_size)
    windows = x.permute(0, 1, 3,5, 2, 4, 6).contiguous().view(-1, window_size, window_size, c_size)
    return windows


def window_reverse_c(windows, window_size,c_size,B, H, W,C):
    x = windows.view(B, H // window_size, W // window_size, C // c_size, window_size, window_size,  c_size)
    x = x.permute(0, 1, 4, 2, 5, 3,6).contiguous().view(B, H, W, C)
    return x

In [37]:
x=torch.rand((2,32,32,128))
windows=window_partition_c(x,2,8)
windows=window_reverse_c(windows, 2,8,2, 32,32,128)

In [38]:
windows.shape

torch.Size([2, 32, 32, 128])

In [12]:
windows.shape

torch.Size([128, 64, 64, 3])

In [13]:
y.shape

torch.Size([2, 32, 32, 256])

In [44]:
#终版
class MemModule_window(nn.Module):
    def __init__(self, mem_dim, fea_dim,window,c_num, shrink_thres=0.0025, device='cuda'):
        super(MemModule_window, self).__init__()
        self.mem_dim = mem_dim
        self.fea_dim = fea_dim
        self.shrink_thres = shrink_thres
        self.memory = MemoryUnit(self.mem_dim, self.fea_dim, self.shrink_thres)
        self.window=window
        self.c_num=c_num
    def forward(self, input):
        s = input.data.shape
        x = input.permute(0, 2, 3, 1).contiguous()
        c_size=s[1]//self.c_num
        if self.c_num==1:
            x=window_partition(x,self.window)
            num_window=(s[2]//self.window)*(s[3]//self.window)
            x=x.view(s[0]*num_window,-1)

            y_and = self.memory(x)
            y = y_and['output']
            att = y_and['att']
            y = window_reverse(y,self.window,s[2],s[3])
            y = y.permute(0, 3, 1, 2)
            print(att.shape)
            att=att.view(s[0], s[2]//self.window, s[3]//self.window, self.mem_dim).permute(0, 3, 1, 2)
            print(att.shape)
            return {'output': y, 'att': att}
        else:
            x=window_partition_c(x,self.window,c_size)
            num_window=(s[2]//self.window)*(s[3]//self.window)*self.c_num
            x=x.view(s[0]*num_window,-1)

            y_and = self.memory(x)
            y = y_and['output']
            att = y_and['att']
            y = window_reverse_c(y,self.window,c_size,s[0],s[2],s[3],s[1])#windows, window_size,c_size,B, H, W,C
            y = y.permute(0, 3, 1, 2)
            att=att.view(s[0], s[2]//self.window, s[3]//self.window,self.c_num, self.mem_dim).permute(0,4,1,2,3)
            print(att.shape)
            
            return {'output': y, 'att': att}


In [46]:
# window 窗口大小 c_num，通道比例 fea_dim=c*w*w/c_num

model1=MemModule_window(mem_dim=2000, fea_dim=512,window=2,c_num=2)
x1=torch.rand((2,256,32,32))
out1=model1(x1)

torch.Size([2, 2000, 16, 16, 2])


In [74]:
out1['output'].shape,out1['att'].shape

(torch.Size([2, 256, 32, 32]), torch.Size([8192, 2000]))

In [73]:
model1=MemModule_window(mem_dim=2000, fea_dim=64,window=2,c_num=16)
x1=torch.rand((2,256,32,32))
out1=model1(x1)

In [41]:
out1.shape

AttributeError: 'dict' object has no attribute 'shape'

In [18]:
class MemModule_ori(nn.Module):
    def __init__(self, mem_dim, fea_dim, shrink_thres=0.0025, device='cuda'):
        super(MemModule_ori, self).__init__()
        self.mem_dim = mem_dim
        self.fea_dim = fea_dim
        self.shrink_thres = shrink_thres
        self.memory = MemoryUnit(self.mem_dim, self.fea_dim, self.shrink_thres)

    def forward(self, input):
        s = input.data.shape
        x = input.permute(0, 2, 3, 1).contiguous()
        print(x.shape)
        x = x.view(-1, s[1])
        print(x.shape)
        #
        y_and = self.memory(x)
        
        #
        y = y_and['output']
        print('y',y.shape)
        att = y_and['att']
        print('att',att.shape)
        
        y = y.view(s[0], s[2], s[3], s[1])
#         print( y.shape)
        y = y.permute(0, 3, 1, 2)
#         print( y.shape)
        att = att.view(s[0], s[2], s[3], self.mem_dim)
#         print( att.shape)
        att = att.permute(0, 3, 1, 2)
#         print( att.shape)

        return {'output': y, 'att': att}



In [19]:
model0=MemModule_ori(mem_dim=2000, fea_dim=256)
x0=torch.rand((2,256,32,32))
out0=model0(x0)

torch.Size([2, 32, 32, 256])
torch.Size([2048, 256])
y torch.Size([2048, 256])
att torch.Size([2048, 2000])


In [4]:
def hard_shrink_relu(input, lambd=0, epsilon=1e-12):
    output = (F.relu(input-lambd) * input) / (torch.abs(input - lambd) + epsilon)
    return output


class MemoryUnit(nn.Module):
    def __init__(self, mem_dim, fea_dim, shrink_thres=0.0025):
        super(MemoryUnit, self).__init__()
        self.mem_dim = mem_dim
        self.fea_dim = fea_dim
        self.weight = Parameter(torch.Tensor(self.mem_dim, self.fea_dim))  # M x C
        self.bias = None
        self.shrink_thres= shrink_thres
        # self.hard_sparse_shrink_opt = nn.Hardshrink(lambd=shrink_thres)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input):
        att_weight = F.linear(input, self.weight)  # Fea x Mem^T, (TxC) x (CxM) = TxM
        att_weight = F.softmax(att_weight, dim=1)  # TxM
        # ReLU based shrinkage, hard shrinkage for positive value
        if(self.shrink_thres>0):
            att_weight = hard_shrink_relu(att_weight, lambd=self.shrink_thres)
#             att_weight = F.softshrink(att_weight, lambd=self.shrink_thres)
            # normalize???
            att_weight = F.normalize(att_weight, p=1, dim=1)
            # att_weight = F.softmax(att_weight, dim=1)
            # att_weight = self.hard_sparse_shrink_opt(att_weight)
        mem_trans = self.weight.permute(1, 0)  # Mem^T, MxC
        output = F.linear(att_weight, mem_trans)  # AttWeight x Mem^T^T = AW x Mem, (TxM) x (MxC) = TxC
        return {'output': output, 'att': att_weight}  # output, att_weight

    def extra_repr(self):
        return 'mem_dim={}, fea_dim={}'.format(
            self.mem_dim, self.fea_dim is not None
        )